# Autoencoder

## Mock dataset

In [1]:
import numpy as np

In [2]:
mappa1="................~............@@@\n##....X........~.....e.......@@@\n###..GJ.........~.@@g$$.....@@.@\n..#...........~~..@..$..@@@@@@@@\n.#.....&......~~..@@@$..@@@@@@@@\n...#...........~.@@@@@@@@@@@@@@@\n.....#...........@@@@@@.@@@@@@@@\n...###........$...@.@@@#.@@@@@@@\n....###@..#.......@@@....@@@@...\n...##@@@@.##......@....##@@.....\n.....@@@@@##...@@@@......@@.....\n.@@@@@@@@@#@@@@@@@@.@..##..#....\n.@@@@@@@@@@.@@@@@@@@@..###.x.&..\n..@@.@.@@@..@@@@@@@@@..##.......\n...@.@@@@@@@@....@@@@#..#...#...\n..@@.@@@@@@@@@..........#####...\n.@@@@@@@@@.@@.........##.#.#.#..\n.@@@@@@@@@..&.....c..#########..\n.@@@@@@@@@......a.f.#.#.#.###...\n..@@@K.............h.########...\n...@@..H.........b...##.######..\n..@@@............$$.##..#######.\n....&......#........########.##.\n..B.........#........##.#..##.#.\n......D...#.#...#....#..####.#..\n$$.............#.i...#.....##...\n...........##....#.##..#..#.....\n..$A.......#.##..##...#....@@@..\n...C.I.....##.....##.#.#...@@@..\n..EF.......##....j..###...@@@@@@\n...........##.......##......@@@.\n................#...##.#...@@@@@"
my_symbol="a"
enemies="upper"

In [3]:
mappa1_processed=[]
for elem in mappa1.splitlines():
    line=[]
    for symbol in elem:
        line.append(symbol)
    mappa1_processed.append(line)

mappa1_processed=np.array(mappa1_processed)


In [4]:
mappa1_processed

array([['.', '.', '.', ..., '@', '@', '@'],
       ['#', '#', '.', ..., '@', '@', '@'],
       ['#', '#', '#', ..., '@', '.', '@'],
       ...,
       ['.', '.', 'E', ..., '@', '@', '@'],
       ['.', '.', '.', ..., '@', '@', '.'],
       ['.', '.', '.', ..., '@', '@', '@']], dtype='<U1')

In [5]:
dataset_raw=[(mappa1_processed,my_symbol,enemies),(mappa1_processed,my_symbol,enemies),(mappa1_processed,my_symbol,enemies),(mappa1_processed,my_symbol,enemies),(mappa1_processed,my_symbol,enemies)]

## Process dataset

In [6]:
dataset_processed=[]
for image,current_symbol,current_enemies in dataset_raw:
    this_image=[]
    for row in range(len(image[0])):
        this_row=[]
        for column in range(len(image[1])):
            current_cell=image[row][column]

            #Immettere infor per i bonus
            if(current_cell == "#" or current_cell == "@" or current_cell == "!" or current_cell == "&"):
                result = 0.8
            elif(current_cell=="X"):
                if(enemies=="upper"):
                    result=0.2
                else:
                    result=0.4
            elif(current_cell=="x"):
                if(enemies=="upper"):
                    result=0.4
                else:
                    result=0.2
            elif(current_cell==current_symbol):
                result=0
            elif(current_cell.isupper() and current_enemies=="upper"):
                result = 0.6
            elif(current_cell.islower() and current_enemies=="lower"):
                result = 0.6 
            else:
                result = 0.0
            this_row.append(result)
        this_image.append(this_row)
    dataset_processed.append(np.array(this_image))
dataset_processed=np.array(dataset_processed)

In [7]:
dataset_processed[1]

array([[0. , 0. , 0. , ..., 0.8, 0.8, 0.8],
       [0.8, 0.8, 0. , ..., 0.8, 0.8, 0.8],
       [0.8, 0.8, 0.8, ..., 0.8, 0. , 0.8],
       ...,
       [0. , 0. , 0.6, ..., 0.8, 0.8, 0.8],
       [0. , 0. , 0. , ..., 0.8, 0.8, 0. ],
       [0. , 0. , 0. , ..., 0.8, 0.8, 0.8]])

## Autoencoder

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

latent_dim = 64 

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(1024, activation='sigmoid'),
      layers.Reshape((32, 32))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [9]:
autoencoder=Autoencoder(latent_dim)
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [10]:

x_train=dataset_processed
x_test=dataset_processed
print(type(x_test))

print (x_train.shape)
print (x_test.shape)

<class 'numpy.ndarray'>
(5, 32, 32)
(5, 32, 32)


In [11]:
autoencoder.fit(x_train, x_train,
                epochs=1000,
                shuffle=True,
                validation_data=(x_test, x_test))

23ms/step - loss: 3.0981e-06 - val_loss: 3.0917e-06
Epoch 818/1000
1/1 [==============================] - 0s 28ms/step - loss: 3.0917e-06 - val_loss: 3.0853e-06
Epoch 819/1000
1/1 [==============================] - 0s 25ms/step - loss: 3.0853e-06 - val_loss: 3.0789e-06
Epoch 820/1000
1/1 [==============================] - 0s 21ms/step - loss: 3.0789e-06 - val_loss: 3.0726e-06
Epoch 821/1000
1/1 [==============================] - 0s 27ms/step - loss: 3.0726e-06 - val_loss: 3.0662e-06
Epoch 822/1000
1/1 [==============================] - 0s 31ms/step - loss: 3.0662e-06 - val_loss: 3.0599e-06
Epoch 823/1000
1/1 [==============================] - 0s 25ms/step - loss: 3.0599e-06 - val_loss: 3.0536e-06
Epoch 824/1000
1/1 [==============================] - 0s 25ms/step - loss: 3.0536e-06 - val_loss: 3.0473e-06
Epoch 825/1000
1/1 [==============================] - 0s 24ms/step - loss: 3.0473e-06 - val_loss: 3.0411e-06
Epoch 826/1000
1/1 [==============================] - 0s 26ms/step - loss: 3

In [12]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()


In [13]:
len(encoded_imgs[0])

64

In [14]:
def post_process_image(decoded_image):
    list_values=[0,0.2,0.4,0.6,0.8]

    for row in range(len(decoded_image[0])):
        for column in range(len(decoded_image[1])):
            current_cell=decoded_image[row][column]
            differences=[abs(current_cell-elem)for elem in list_values]
            decoded_image[row][column]=list_values[differences.index(min(differences))]
    return decoded_image



In [15]:
post_process_image(decoded_imgs[0])

array([[0. , 0. , 0. , ..., 0.8, 0.8, 0.8],
       [0.8, 0.8, 0. , ..., 0.8, 0.8, 0.8],
       [0.8, 0.8, 0.8, ..., 0.8, 0. , 0.8],
       ...,
       [0. , 0. , 0.6, ..., 0.8, 0.8, 0.8],
       [0. , 0. , 0. , ..., 0.8, 0.8, 0. ],
       [0. , 0. , 0. , ..., 0.8, 0.8, 0.8]], dtype=float32)

## Mock statset sequences

In [16]:
sequence=[dataset_processed[0],dataset_processed[1],dataset_processed[2]]
dataset_sequences=np.array([sequence,sequence,sequence])
encoded_sequences=[]
for sequence_one in dataset_sequences:
    encoded_imgs = autoencoder.encoder(sequence_one).numpy()
    encoded_sequences.append(encoded_imgs)
x_train=np.array(encoded_sequences)
y_train=np.array(encoded_sequences)

In [17]:
encoded_sequences

[array([[ 0.      ,  8.985762,  0.      ,  9.236497,  7.241758,  0.      ,
          0.      ,  0.      ,  0.      ,  9.071447,  7.774665,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      , 10.020109,  0.      ,  9.851858,  9.137993,  0.      ,
         10.253996,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  9.378804,  0.      ,  0.      ,
          0.      ,  8.063891,  9.001828,  0.      ,  0.      ,  9.984906,
          9.49766 ,  7.328406,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  9.649704,  0.      ,  0.      ,  0.      ,  0.      ,
          8.987841,  9.553394,  0.      ,  0.      ,  0.      ,  8.790589,
         10.647666,  8.029093,  0.      ,  9.269299],
        [ 0.      ,  8.985762,  0.      ,  9.236497,  7.241758,  0.      ,
          0.      ,  0.      ,  0.      ,  9.071447,  7.774665,  0.      ,
          0.      ,  0.      ,  0.      ,  0. 

# Lstm

In [69]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
lstm_model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(3, 64)),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(units=64)
])

In [19]:
lstm_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mse'])

In [20]:
lstm_model.fit(x=x_train, y=y_train,epochs=5)

Epoch 1/5
1/1 [==============================] - 0s 998us/step - loss: 28.8813 - mse: 28.8813
Epoch 2/5
1/1 [==============================] - 0s 2ms/step - loss: 28.3894 - mse: 28.3894
Epoch 3/5
1/1 [==============================] - 0s 3ms/step - loss: 27.7312 - mse: 27.7312
Epoch 4/5
1/1 [==============================] - 0s 3ms/step - loss: 27.4835 - mse: 27.4835
Epoch 5/5
1/1 [==============================] - 0s 2ms/step - loss: 27.2572 - mse: 27.2572


In [62]:
predicted=np.array(lstm_model.predict(x_train))

In [63]:
len(predicted)

3

In [64]:
post_process_image(autoencoder.decoder(predicted).numpy()[0])

array([[0.4, 0.4, 0.4, ..., 0.6, 0.4, 0.4],
       [0.4, 0.4, 0.4, ..., 0.6, 0.4, 0.6],
       [0.6, 0.4, 0.6, ..., 0.6, 0.4, 0.6],
       ...,
       [0.4, 0.4, 0.4, ..., 0.6, 0.6, 0.6],
       [0.4, 0.4, 0.4, ..., 0.6, 0.6, 0.4],
       [0.4, 0.4, 0.6, ..., 0.4, 0.6, 0.6]], dtype=float32)

In [65]:
decode_predicted=autoencoder.decoder(predicted).numpy()

In [66]:
post_processed_prediction=[]
for elem in decode_predicted:
    current=post_process_image(elem)
    post_processed_prediction.append(current)

In [67]:
print(post_processed_prediction)

[array([[0.4, 0.4, 0.4, ..., 0.6, 0.4, 0.4],
       [0.4, 0.4, 0.4, ..., 0.6, 0.4, 0.6],
       [0.6, 0.4, 0.6, ..., 0.6, 0.4, 0.6],
       ...,
       [0.4, 0.4, 0.4, ..., 0.6, 0.6, 0.6],
       [0.4, 0.4, 0.4, ..., 0.6, 0.6, 0.4],
       [0.4, 0.4, 0.6, ..., 0.4, 0.6, 0.6]], dtype=float32), array([[0.4, 0.4, 0.4, ..., 0.6, 0.4, 0.4],
       [0.4, 0.4, 0.4, ..., 0.6, 0.4, 0.6],
       [0.6, 0.4, 0.6, ..., 0.6, 0.4, 0.6],
       ...,
       [0.4, 0.4, 0.4, ..., 0.6, 0.6, 0.6],
       [0.4, 0.4, 0.4, ..., 0.6, 0.6, 0.4],
       [0.4, 0.4, 0.6, ..., 0.4, 0.6, 0.6]], dtype=float32), array([[0.4, 0.4, 0.4, ..., 0.6, 0.4, 0.4],
       [0.4, 0.4, 0.4, ..., 0.6, 0.4, 0.6],
       [0.6, 0.4, 0.6, ..., 0.6, 0.4, 0.6],
       ...,
       [0.4, 0.4, 0.4, ..., 0.6, 0.6, 0.6],
       [0.4, 0.4, 0.4, ..., 0.6, 0.6, 0.4],
       [0.4, 0.4, 0.6, ..., 0.4, 0.6, 0.6]], dtype=float32)]
